In [1]:
import numpy as np
import pandas as pd 

from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# defining dtypes upon reading in data to keep FIPS as 5 digit code
dtypes_dict = {
 'FIPS': 'object',
 'State': 'object',
 'County': 'object',
 'Deaths': 'float64',
 'Years of Potential Life Lost Rate': 'float64',
 '% Fair or Poor Health': 'float64',
 'Average Number of Physically Unhealthy Days': 'float64',
 'Average Number of Mentally Unhealthy Days': 'float64',
 '% Low birthweight': 'float64',
 '% Smokers': 'float64',
 '% Adults with Obesity': 'float64',
 'Food Environment Index': 'float64',
 '% Physically Inactive': 'float64',
 '% With Access to Exercise Opportunities': 'float64',
 '% Excessive Drinking': 'float64',
 '# Alcohol-Impaired Driving Deaths': 'float64',
 '# Driving Deaths': 'float64',
 '% Driving Deaths with Alcohol Involvement': 'float64',
 '# Chlamydia Cases': 'float64',
 'Chlamydia Rate': 'float64',
 'Teen Birth Rate': 'float64',
 '# Uninsured': 'float64',
 '% Uninsured': 'float64',
 '# Primary Care Physicians': 'float64',
 'Primary Care Physicians Rate': 'float64',
 'Primary Care Physicians Ratio': 'object',
 '# Dentists': 'float64',
 'Dentist Rate': 'float64',
 'Dentist Ratio': 'object',
 '# Mental Health Providers': 'float64',
 'Mental Health Provider Rate': 'float64',
 'Mental Health Provider Ratio': 'object',
 'Preventable Hospitalization Rate': 'float64',
 '% With Annual Mammogram': 'float64',
 '% Vaccinated': 'float64',
 '# Completed High School': 'int64',
 'Population': 'int64',
 '% Completed High School': 'float64',
 '# Some College': 'int64',
 'Population.1': 'int64',
 '% Some College': 'float64',
 '# Unemployed': 'float64',
 'Labor Force': 'float64',
 '% Unemployed': 'float64',
 '% Children in Poverty': 'float64',
 'Income Ratio': 'float64',
 '# Children in Single-Parent Households': 'float64',
 '# Children in Households': 'float64',
 '% Children in Single-Parent Households': 'float64',
 '# Associations': 'int64',
 'Social Association Rate': 'float64',
 'Annual Average Violent Crimes': 'float64',
 'Violent Crime Rate': 'float64',
 '# Injury Deaths': 'float64',
 'Injury Death Rate': 'float64',
 'Average Daily PM2.5': 'float64',
 '% Severe Housing Problems': 'float64',
 'Severe Housing Cost Burden': 'float64',
 'Overcrowding': 'float64',
 'Inadequate Facilities': 'float64',
 '% Drive Alone to Work': 'float64',
 '# Workers who Drive Alone': 'int64',
 '% Long Commute - Drives Alone': 'float64',
 'Water_Violation': 'int64'}

In [3]:
police = pd.read_csv('../data/police_scorecard_clean.csv', dtype={'location_name': object, 
                                                                  'state': object,
                                                                  'calc_overall_score': np.int64, 
                                                                  'FIPS': object})
mei = pd.read_csv('../data/mei_clean.csv')
ch_rankings = pd.read_csv('../data/chr_clean.csv', dtype=dtypes_dict)

In [4]:
ch_rankings.head()

,FIPS,State,County,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low birthweight,% Smokers,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,# Uninsured,% Uninsured,# Primary Care Physicians,Primary Care Physicians Rate,Primary Care Physicians Ratio,# Dentists,Dentist Rate,Dentist Ratio,# Mental Health Providers,Mental Health Provider Rate,Mental Health Provider Ratio,Preventable Hospitalization Rate,% With Annual Mammogram,% Vaccinated,# Completed High School,Population,% Completed High School,# Some College,Population.1,% Some College,# Unemployed,Labor Force,% Unemployed,% Children in Poverty,Income Ratio,# Children in Single-Parent Households,# Children in Households,% Children in Single-Parent Households,# Associations,Social Association Rate,Annual Average Violent Crimes,Violent Crime Rate,# Injury Deaths,Injury Death Rate,Average Daily PM2.5,% Severe Housing Problems,Severe Housing Cost Burden,Overcrowding,Inadequate Facilities,% Drive Alone to Work,# Workers who Drive Alone,% Long Commute - Drives Alone,Water_Violation
0,01001,Alabama,Autauga,787.0,7830.053484,19.839179,4.501499,4.850214,9.201774,19.810766,33.0,6.7,30.6,69.130124,14.473886,19.0,57.0,33.333333,280.0,504.5,23.764041,4693.0,10.047314,26.0,46.76175,2139:1,19.0,34.00813,2940:1,16.0,28.63842,3492:1,6650.0,39.0,42.0,33076,37367,88.516606,8689,14362,60.499930,714.0,26172.0,2.728106,15.9,5.090408,3267.0,13205.0,24.740629,68,12.229996,148.5,272.282220,190.0,68.410043,10.4,13.630967,12.032598,1.116124,1.543576,85.788512,24635,38.3,0
1,01003,Alabama,Baldwin,3147.0,7680.477270,16.460675,3.647978,4.768294,8.292009,18.513318,30.0,7.8,24.7,73.713549,18.736958,53.0,179.0,29.608939,632.0,297.2,26.006904,22660.0,13.168983,153.0,70.17640,1425:1,110.0,49.27565,2029:1,220.0,98.55130,1015:1,3471.0,43.0,46.0,137219,151112,90.806157,33404,49776,67.108647,2653.0,97328.0,2.725834,13.5,4.385135,10321.0,46133.0,22.372271,229,10.503527,408.0,203.660396,777.0,72.878641,7.2,12.996651,11.987507,1.017929,0.334931,83.760106,93141,40.4,0
2,01005,Alabama,Barbour,515.0,11476.629416,29.841500,5.569267,5.590494,11.355311,25.530768,41.2,5.5,28.0,53.166770,12.466974,13.0,33.0,39.393939,191.0,755.8,37.114262,2310.0,13.504034,8.0,32.15305,3110:1,9.0,36.45791,2743:1,3.0,12.15264,8229:1,5314.0,44.0,39.0,13152,17964,73.213093,2379,6680,35.613772,324.0,8537.0,3.795244,41.0,5.981993,2707.0,5307.0,51.008103,19,7.636349,105.5,414.277861,98.0,76.988947,9.4,13.710555,12.594260,1.958651,0.652884,82.116389,8231,30.9,0
3,01007,Alabama,Bibb,476.0,12172.562382,23.853284,4.894377,5.271114,10.220994,23.084813,37.4,7.6,33.4,16.251364,15.420603,8.0,27.0,29.629630,139.0,613.2,37.786446,1764.0,10.599051,12.0,53.57143,1867:1,5.0,22.32741,4479:1,6.0,26.79289,3732:1,6690.0,33.0,40.0,12782,16168,79.057397,2483,6186,40.139024,266.0,8685.0,3.062752,25.9,5.002608,1404.0,4586.0,30.614915,19,8.482143,19.5,89.349126,119.0,105.601306,10.0,9.255242,8.263736,0.563991,0.563991,87.057671,8167,52.0,0
4,01009,Alabama,Blount,1100.0,11096.050649,21.985610,4.986622,5.357809,7.430541,22.954959,33.0,7.9,33.3,15.634486,15.833284,15.0,82.0,18.292683,176.0,303.4,31.239778,6642.0,14.115997,12.0,20.74689,4820:1,11.0,19.02258,5257:1,10.0,17.29326,5783:1,4440.0,37.0,40.0,32028,39791,80.490563,7862,14102,55.750957,676.0,25331.0,2.668667,21.0,4.427838,3003.0,13372.0,22.457374,45,7.780083,279.0,482.690611,311.0,107.591609,10.6,10.125665,7.468859,1.812470,1.159981,87.519594,21690,57.9,0


In [5]:
police.head()

,location_name,state,calc_overall_score,FIPS
0,new york,NY,32,36061
1,los angeles,CA,33,06037
2,chicago,IL,24,17031
3,houston,TX,48,48201
4,phoenix,AZ,39,04013


In [6]:
police.dtypes

location_name         object
state                 object
calc_overall_score     int64
FIPS                  object
dtype: object

In [7]:
police['location_name'].nunique()

10368

In [8]:
mei.head()

,state,city,standard_score,flex_score,total_mei
0,AL,auburn,21,2,23
1,AL,birmingham,94,8,100
2,AL,florence,0,0,0
3,AL,hoover,12,0,12
4,AL,huntsville,20,0,20


In [9]:
mei.shape

(506, 5)

In [10]:
mei.dtypes

state             object
city              object
standard_score     int64
flex_score         int64
total_mei          int64
dtype: object

In [11]:
ch_rankings.head()

,FIPS,State,County,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low birthweight,% Smokers,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,# Uninsured,% Uninsured,# Primary Care Physicians,Primary Care Physicians Rate,Primary Care Physicians Ratio,# Dentists,Dentist Rate,Dentist Ratio,# Mental Health Providers,Mental Health Provider Rate,Mental Health Provider Ratio,Preventable Hospitalization Rate,% With Annual Mammogram,% Vaccinated,# Completed High School,Population,% Completed High School,# Some College,Population.1,% Some College,# Unemployed,Labor Force,% Unemployed,% Children in Poverty,Income Ratio,# Children in Single-Parent Households,# Children in Households,% Children in Single-Parent Households,# Associations,Social Association Rate,Annual Average Violent Crimes,Violent Crime Rate,# Injury Deaths,Injury Death Rate,Average Daily PM2.5,% Severe Housing Problems,Severe Housing Cost Burden,Overcrowding,Inadequate Facilities,% Drive Alone to Work,# Workers who Drive Alone,% Long Commute - Drives Alone,Water_Violation
0,01001,Alabama,Autauga,787.0,7830.053484,19.839179,4.501499,4.850214,9.201774,19.810766,33.0,6.7,30.6,69.130124,14.473886,19.0,57.0,33.333333,280.0,504.5,23.764041,4693.0,10.047314,26.0,46.76175,2139:1,19.0,34.00813,2940:1,16.0,28.63842,3492:1,6650.0,39.0,42.0,33076,37367,88.516606,8689,14362,60.499930,714.0,26172.0,2.728106,15.9,5.090408,3267.0,13205.0,24.740629,68,12.229996,148.5,272.282220,190.0,68.410043,10.4,13.630967,12.032598,1.116124,1.543576,85.788512,24635,38.3,0
1,01003,Alabama,Baldwin,3147.0,7680.477270,16.460675,3.647978,4.768294,8.292009,18.513318,30.0,7.8,24.7,73.713549,18.736958,53.0,179.0,29.608939,632.0,297.2,26.006904,22660.0,13.168983,153.0,70.17640,1425:1,110.0,49.27565,2029:1,220.0,98.55130,1015:1,3471.0,43.0,46.0,137219,151112,90.806157,33404,49776,67.108647,2653.0,97328.0,2.725834,13.5,4.385135,10321.0,46133.0,22.372271,229,10.503527,408.0,203.660396,777.0,72.878641,7.2,12.996651,11.987507,1.017929,0.334931,83.760106,93141,40.4,0
2,01005,Alabama,Barbour,515.0,11476.629416,29.841500,5.569267,5.590494,11.355311,25.530768,41.2,5.5,28.0,53.166770,12.466974,13.0,33.0,39.393939,191.0,755.8,37.114262,2310.0,13.504034,8.0,32.15305,3110:1,9.0,36.45791,2743:1,3.0,12.15264,8229:1,5314.0,44.0,39.0,13152,17964,73.213093,2379,6680,35.613772,324.0,8537.0,3.795244,41.0,5.981993,2707.0,5307.0,51.008103,19,7.636349,105.5,414.277861,98.0,76.988947,9.4,13.710555,12.594260,1.958651,0.652884,82.116389,8231,30.9,0
3,01007,Alabama,Bibb,476.0,12172.562382,23.853284,4.894377,5.271114,10.220994,23.084813,37.4,7.6,33.4,16.251364,15.420603,8.0,27.0,29.629630,139.0,613.2,37.786446,1764.0,10.599051,12.0,53.57143,1867:1,5.0,22.32741,4479:1,6.0,26.79289,3732:1,6690.0,33.0,40.0,12782,16168,79.057397,2483,6186,40.139024,266.0,8685.0,3.062752,25.9,5.002608,1404.0,4586.0,30.614915,19,8.482143,19.5,89.349126,119.0,105.601306,10.0,9.255242,8.263736,0.563991,0.563991,87.057671,8167,52.0,0
4,01009,Alabama,Blount,1100.0,11096.050649,21.985610,4.986622,5.357809,7.430541,22.954959,33.0,7.9,33.3,15.634486,15.833284,15.0,82.0,18.292683,176.0,303.4,31.239778,6642.0,14.115997,12.0,20.74689,4820:1,11.0,19.02258,5257:1,10.0,17.29326,5783:1,4440.0,37.0,40.0,32028,39791,80.490563,7862,14102,55.750957,676.0,25331.0,2.668667,21.0,4.427838,3003.0,13372.0,22.457374,45,7.780083,279.0,482.690611,311.0,107.591609,10.6,10.125665,7.468859,1.812470,1.159981,87.519594,21690,57.9,0


In [12]:
ch_rankings.shape

(3141, 64)

In [13]:
df = pd.merge(police, mei, how='right', 
              left_on=['location_name', 'state'], right_on=['city', 'state'])

In [14]:
df.shape

(510, 8)

In [15]:
df.isnull().sum()

location_name         75
state                  0
calc_overall_score    75
FIPS                  75
city                   0
standard_score         0
flex_score             0
total_mei              0
dtype: int64

In [16]:
df.dropna(inplace=True)

In [17]:
df.head()

,location_name,state,calc_overall_score,FIPS,city,standard_score,flex_score,total_mei
0,auburn,AL,42.0,01081,auburn,21,2,23
1,birmingham,AL,43.0,01073,birmingham,94,8,100
2,florence,AL,40.0,01077,florence,0,0,0
3,hoover,AL,44.0,01073,hoover,12,0,12
4,huntsville,AL,46.0,01089,huntsville,20,0,20


In [18]:
# df['fips_county_code'] = df['fips_county_code'].astype(int)
df['calc_overall_score'] = df['calc_overall_score'].astype(int)

In [19]:
df = pd.merge(df, ch_rankings, how='left', 
         left_on='FIPS', right_on='FIPS')

In [20]:
df.shape

(435, 71)

In [21]:
df.columns[df.isna().any()].tolist()

['Food Environment Index',
 'Preventable Hospitalization Rate',
 '% With Annual Mammogram',
 '% Vaccinated',
 'Annual Average Violent Crimes',
 'Violent Crime Rate',
 'Average Daily PM2.5']

In [22]:
df.describe()

,calc_overall_score,standard_score,flex_score,total_mei,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low birthweight,% Smokers,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,# Uninsured,% Uninsured,# Primary Care Physicians,Primary Care Physicians Rate,# Dentists,Dentist Rate,# Mental Health Providers,Mental Health Provider Rate,Preventable Hospitalization Rate,% With Annual Mammogram,% Vaccinated,# Completed High School,Population,% Completed High School,# Some College,Population.1,% Some College,# Unemployed,Labor Force,% Unemployed,% Children in Poverty,Income Ratio,# Children in Single-Parent Households,# Children in Households,% Children in Single-Parent Households,# Associations,Social Association Rate,Annual Average Violent Crimes,Violent Crime Rate,# Injury Deaths,Injury Death Rate,Average Daily PM2.5,% Severe Housing Problems,Severe Housing Cost Burden,Overcrowding,Inadequate Facilities,% Drive Alone to Work,# Workers who Drive Alone,% Long Commute - Drives Alone,Water_Violation
count,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,434.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,434.000000,434.000000,434.000000,4.350000e+02,4.350000e+02,435.000000,4.350000e+02,4.350000e+02,435.000000,435.000000,4.350000e+02,435.000000,435.000000,435.000000,435.000000,4.350000e+02,435.000000,435.000000,435.000000,431.000000,431.000000,435.000000,435.000000,433.000000,435.000000,435.000000,435.000000,435.000000,435.000000,4.350000e+02,435.000000,435.000000
mean,44.786207,60.650575,4.422989,63.829885,9115.855172,6861.056238,17.621227,4.001297,4.361272,8.082824,17.141800,29.284138,7.935023,21.281839,87.904528,19.414151,105.218391,396.710345,28.569387,5695.154023,584.038391,19.735131,87112.406897,10.006514,770.940230,87.700819,784.404598,78.368561,2868.760920,320.656537,3997.986175,42.771889,49.451613,5.381934e+05,6.266923e+05,89.365700,1.781894e+05,2.645957e+05,68.713584,17467.558621,4.787429e+05,3.523391,15.931494,4.748562,54142.094253,2.109606e+05,25.773798,662.535632,9.765989,4150.726218,403.376498,2818.243678,72.612729,8.218245,17.806902,14.741836,3.216277,1.051555,77.450207,4.477422e+05,30.585287,0.004598
std,6.889034,28.470086,4.521370,29.908671,13906.428754,1924.977647,3.975897,0.556506,0.526717,1.480801,3.502330,5.239754,0.788819,4.489210,10.339473,3.025919,160.756408,634.468978,8.087510,10386.729614,221.952119,9.448062,163748.984509,4.406453,1203.133199,31.708065,1403.524929,25.751695,5343.051066,168.922715,1096.811836,6.493631,6.764171,8.742791e+05,1.075828e+06,4.776734,3.032125e+05,4.671734e+05,8.154469,33949.400428,8.091553e+05,0.981177,6.169047,0.725179,90976.244167,3.571721e+05,7.272161,916.143752,3.698258,7822.522791,235.069489,3958.648914,21.800056,1.997954,4.799517,3.666992,2.479282,0.741545,7.966473,7.564714e+05,12.154005,0.067728
min,24.000000,0.000000,0.000000,0.000000,98.000000,2872.671890,9.847407,2.667134,2.974246,4.929721,7.075650,14.300000,2.800000,8.900000,43.281766,6.453001,1.000000,3.000000,8.024691,46.000000,130.400000,2.098695,1019.000000,2.764273,7.000000,31.169580,8.000000,24.335570,7.000000,25.506950,455.000000,27.000000,26.000000,6.603000e+03,6.953000e+03,65.721320,2.241000e+03,2.878000e+03,43.647279,235.000000,6.884000e+03,1.791881,3.600000,3.523520,562.000000,2.352000e+03,9.157569,16.000000,1.816098,11.000000,42.286840,29.000000,30.834150,3.900000,8.424818,6.047820,0.358491,0.000000,5.990142,7.104000e+03,5.100000,0.000000
25%,40.000000,41.500000,0.000

In [23]:
# Imputing Alexandria, VA Food Environment Index (from 2017) 8.7
# source: http://www.healthmattersalexandria.org/indicators/index/view?indicatorId=2362&localeId=2967

df.loc[416, 'Food Environment Index'] = 8.7

In [24]:
# When investigating other null values, Homer, AK was missing many values
# so I removed it instead of imputing multiple values for the same city
df.drop(index=10, inplace=True)

In [25]:
# imputing number of violent crimes per 
# source: https://www.neighborhoodscout.com/

df.loc[9, 'Annual Average Violent Crimes'] = 251
df.loc[12, 'Annual Average Violent Crimes'] = 48
df.loc[331, 'Annual Average Violent Crimes'] = 29

df.drop(columns='Violent Crime Rate', inplace=True)

In [26]:
df.loc[11, 'Average Daily PM2.5'] = 2.1

In [27]:
df.columns[df.isna().any()].tolist()

[]

In [28]:
# need to fix 3 ratio columns
cols_to_fix = ['Primary Care Physicians Ratio', 'Dentist Ratio', 'Mental Health Provider Ratio']

for col in cols_to_fix:
    df[col] = df[col].str.replace(':1', '')
    df[col] = df[col].astype(int)

### Ready to Look at Correlation/Models

In [30]:
df.corr()['standard_score'].sort_values()

% Drive Alone to Work                         -0.347112
Teen Birth Rate                               -0.334926
% Uninsured                                   -0.328484
% Adults with Obesity                         -0.320691
% Physically Inactive                         -0.301323
Primary Care Physicians Ratio                 -0.299463
Dentist Ratio                                 -0.288834
% Smokers                                     -0.242205
Mental Health Provider Ratio                  -0.230495
Years of Potential Life Lost Rate             -0.208716
calc_overall_score                            -0.186174
% Fair or Poor Health                         -0.184155
Average Number of Physically Unhealthy Days   -0.154094
% Driving Deaths with Alcohol Involvement     -0.117975
% Children in Poverty                         -0.116476
Social Association Rate                       -0.101350
Average Number of Mentally Unhealthy Days     -0.095583
% Low birthweight                             -0

In [29]:
df.corr()['calc_overall_score']

calc_overall_score                             1.000000
standard_score                                -0.186174
flex_score                                    -0.167786
total_mei                                     -0.177405
Deaths                                        -0.063295
Years of Potential Life Lost Rate             -0.203956
% Fair or Poor Health                         -0.083703
Average Number of Physically Unhealthy Days   -0.098194
Average Number of Mentally Unhealthy Days     -0.160483
% Low birthweight                             -0.175929
% Smokers                                     -0.147456
% Adults with Obesity                         -0.014285
Food Environment Index                         0.060355
% Physically Inactive                         -0.155367
% With Access to Exercise Opportunities       -0.003536
% Excessive Drinking                          -0.029535
# Alcohol-Impaired Driving Deaths              0.004855
# Driving Deaths                              -0

In [30]:
cols_not_using = ['location_name', 'state', 'FIPS', 'city', 'State', 
                  'County', 'total_mei', 'standard_score', 'flex_score', 'calc_overall_score']

features = [col for col in df.columns if col not in cols_not_using]
X = df[features]
y = df['total_mei']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=73)

lr = LinearRegression()

lr.fit(X_train, y_train)

lr.score(X_train, y_train), lr.score(X_test, y_test)

(0.5489244302403777, 0.2579849738148734)

In [31]:
lasso = LassoCV()

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

lasso.fit(X_train_sc, y_train)
lasso.score(X_train_sc, y_train), lasso.score(X_test_sc, y_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 72.00238435783831, tolerance: 22.835081153846154
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 93.73772892761917, tolerance: 22.835081153846154
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 88.87936232778884, tolerance: 22.835081153846154
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526:

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25.157272306838422, tolerance: 23.698235
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24.316898369579576, tolerance: 23.698235
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 44.52460273774341, tolerance: 23.698235
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Obje

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28.003599100076826, tolerance: 24.655059615384612
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 59.21352600163664, tolerance: 24.655059615384612
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 105.25970471207984, tolerance: 24.655059615384612
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:52

(0.47628095240292134, 0.3700780542871931)

In [32]:
lasso.coef_

array([ 0.        ,  0.        ,  8.95909039, -5.45588094,  3.21055296,
        0.        ,  0.        ,  1.30933903,  2.99609655, -7.26531352,
        7.6184727 ,  0.93532741,  5.28569791,  0.        , -2.23834405,
       -0.        ,  2.34989667, -2.55927128,  0.        , -8.75751843,
       -0.        ,  0.        , -3.59979908, -4.16552678,  0.97681361,
        1.15079305, -1.35219961, -2.12411433, -0.        ,  1.61731733,
       -0.        ,  0.24654513, -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
        0.51189738,  2.48108256, -3.55494061, -0.        , -0.        ,
        0.        , -0.        , -1.01564844, -0.51072044,  2.85842326,
        3.40864757, -0.        ,  0.        ,  5.32476589, -2.07530847,
        0.        , -5.66604458, -0.        , -0.        ,  0.59700912])

## Exploring Clustering

In [33]:
df.columns.values[0]

'location_name'

In [34]:

cols_not_using = ['location_name', 'state', 'FIPS', 'city', 'State', 
                  'County']

features = [col for col in df.columns if col not in cols_not_using]
X = df[features]
ss = StandardScaler()
X_sc = ss.fit_transform(X)

model = KMeans(random_state=42)
model.fit(X_sc)
X_sc = pd.DataFrame(X_sc, columns=features)

In [35]:
labels = model.labels_
centroids = model.cluster_centers_

print("Predicted clusters to points: ", labels)
print("Location of centroids: ")
print(centroids)

Predicted clusters to points:  [6 0 6 0 6 7 7 6 3 3 6 6 1 1 0 1 1 1 1 1 1 1 0 6 6 6 7 7 0 0 6 4 7 2 2 4 4
 2 4 4 4 4 2 7 4 4 5 2 4 4 5 5 7 2 4 4 4 4 4 5 5 2 4 4 4 4 4 2 2 4 5 2 5 4
 2 5 3 7 3 3 2 3 2 3 3 3 2 2 2 2 0 2 2 0 6 6 0 6 0 6 4 6 4 0 4 4 4 4 4 4 0
 0 4 4 0 0 7 0 0 0 0 3 6 3 3 6 6 3 0 1 0 0 3 0 6 7 0 0 6 7 6 3 3 3 3 3 2 6
 3 3 6 6 6 6 2 2 3 0 0 4 4 3 6 2 2 2 2 3 2 0 3 2 0 0 2 3 0 3 3 3 3 3 6 6 3
 3 6 3 6 4 4 4 4 0 6 3 3 3 3 3 3 3 3 2 4 3 3 0 3 3 3 3 6 3 3 0 0 0 0 0 0 0
 0 6 0 7 6 6 0 6 0 0 3 2 0 2 3 2 6 0 6 3 6 0 7 6 0 7 4 2 7 6 7 1 3 7 4 1 1
 1 1 6 7 6 7 3 1 1 3 3 4 7 3 3 6 0 0 7 0 6 0 3 3 6 2 0 3 6 6 6 6 3 6 3 6 6
 7 3 6 6 6 6 6 0 0 7 6 7 0 6 7 7 0 7 0 2 6 3 6 2 3 2 2 2 3 7 7 7 7 7 6 6 7
 6 3 0 6 0 6 3 0 0 3 3 6 6 4 4 3 3 0 4 0 0 7 7 7 6 7 0 3 0 3 2 0 0 2 2 3 2
 3 0 0 0 0 0 0 6 0 6 3 0 0 3 0 0 3 0 6 0 6 3 6 3 6 3 3 3 3 3 3 3 3 3 3 3 3
 3 2 3 3 3 3 2 2 2 2 3 0 0 0 0 0 3 3 2 3 2 3 3 2 3 0 3]
Location of centroids: 
[[-3.41201244e-01  4.21260816e-01  3.12868518e-01  4.37040746e-0

In [36]:
X_sc['labels'] = labels
X_sc.head()

,calc_overall_score,standard_score,flex_score,total_mei,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low birthweight,% Smokers,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Teen Birth Rate,# Uninsured,% Uninsured,# Primary Care Physicians,Primary Care Physicians Rate,Primary Care Physicians Ratio,# Dentists,Dentist Rate,Dentist Ratio,# Mental Health Providers,Mental Health Provider Rate,Mental Health Provider Ratio,Preventable Hospitalization Rate,% With Annual Mammogram,% Vaccinated,# Completed High School,Population,% Completed High School,# Some College,Population.1,% Some College,# Unemployed,Labor Force,% Unemployed,% Children in Poverty,Income Ratio,# Children in Single-Parent Households,# Children in Households,% Children in Single-Parent Households,# Associations,Social Association Rate,Annual Average Violent Crimes,# Injury Deaths,Injury Death Rate,Average Daily PM2.5,% Severe Housing Problems,Severe Housing Cost Burden,Overcrowding,Inadequate Facilities,% Drive Alone to Work,# Workers who Drive Alone,% Long Commute - Drives Alone,Water_Violation,labels
0,-0.403206,-1.403420,-0.537310,-1.375276,-0.526597,0.198582,0.586866,1.033394,1.423416,0.630865,0.616273,0.291103,-1.569317,-0.307856,-1.949761,-0.814387,-0.488146,-0.492787,0.394165,-0.471908,-0.383374,-0.436297,-0.443399,0.122985,-0.572593,-1.151074,1.458395,-0.521797,-1.769811,3.318341,-0.509419,-1.345486,2.022156,0.473748,-1.044054,-0.658876,-0.516210,-0.493731,0.319311,-0.485097,-0.476753,0.670115,-0.454427,-0.498485,-0.825758,-0.282057,2.113888,-0.492388,-0.495934,0.250425,-0.580802,-0.461311,-0.445339,-0.587632,-0.489548,1.288383,0.268524,0.584274,-0.144474,-0.516683,0.702032,-0.495696,-0.133731,-0.068041,6
1,-0.257985,1.172064,0.790162,1.210029,0.159125,2.194468,0.663303,0.243459,0.677362,2.212774,0.379709,1.016750,-1.696124,1.741503,-0.895941,-0.719641,-0.257889,0.049601,-1.689305,-0.056529,0.867797,0.740723,-0.165777,0.257334,-0.046245,0.663106,-0.794679,-0.127661,0.535208,-0.660119,-0.303108,-0.770408,0.320956,0.511173,-0.119006,-0.362859,-0.153656,-0.165789,0.147982,-0.183622,-0.184362,-0.010923,-0.247863,-0.199083,-0.685415,1.063851,0.840721,0.028799,-0.169659,1.640639,0.320120,1.290845,0.192429,0.138974,1.366860,1.288383,-0.313612,0.021019,-0.768000,-0.144657,0.722960,-0.196382,0.253747,-0.068041,0
2,-0.693648,-2.144312,-0.979801,-2.147510,-0.548683,0.917274,0.716607,0.826856,1.400644,1.148326,0.820908,0.577543,-0.681664,1.273715,-2.168398,-1.281330,-0.544154,-0.500671,-0.751650,-0.510143,-0.635958,0.223071,-0.481615,0.388943,-0.604191,-1.196730,1.580655,-0.523222,-0.866493,0.755255,-0.510917,-0.967390,0.668076,1.043327,-0.427355,0.229174,-0.553542,-0.524833,-0.413980,-0.548788,-0.522293,-1.367328,-0.474815,-0.540558,-0.261905,0.723320,-0.275729,-0.548148,-0.541007,-0.223161,-0.577526,1.317153,-0.497176,-0.621751,0.281175,0.692795,-1.293701,-1.327093,-0.811057,-0.440489,1.366370,-0.540903,0.055886,-0.068041,6
3,-0.112764,-1.720945,-0.979801,-1.744605,0.159125,2.194468,0.663303,0.243459,0.677362,2.212774,0.379709,1.016750,-1.696124,1.741503,-0.895941,-0.719641,-0.257889,0.049601,-1.689305,-0.056529,0.867797,0.740723,-0.165777,0.257334,-0.046245,0.663106,-0.794679,-0.127661,0.535208,-0.660119,-0.303108,-0.770408,0.320956,0.511173,-0.119006,-0.362859,-0.153656,-0.165789,0.147982,-0.183622,-0.184362,-0.010923,-0.247863,-0.199083,-0.685415,1.063851,0.840721,0.028799,-0.169659,1.640639,0.320120,1.290845,0.192429,0.138974,1.366860,1.288383,-0.313612,0.021019,-0.768000,-0.144657,0.722960,-0.196382,0.253747,-0.068041,0
4,0.177678,-1.438700,-0.979801,-1.476002,-0.312219,0.638987,0.058165,-0.413849,0.158223,1.175607,-0.011047,0.863982,-0.554856,0.672273,-1.

In [37]:
score = silhouette_score(X_sc, labels, metric='euclidean')
score

0.1866657773708327